In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import numpy as np
from joblib import dump, load

In [2]:
opath = "../../cunha/covid19-sample/"

df = pd.read_csv(opath+"training.csv", sep=';', on_bad_lines='skip')
df

,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,retweet_count,country_code,place_full_name,place_type,followers_count,friends_count,account_lang,account_created_at,verified,lang
0,1244776527499149315,80795043,2020-03-31T00:00:25Z,SimonWeaving,What colour is that virus? #coronavirus https:...,Twitter for iPhone,NaN,NaN,NaN,False,...,0,AU,"Newcastle, New South Wales",city,542,447,NaN,2009-10-08T07:50:31Z,False,en
1,1244776614900056064,3311560502,2020-03-31T00:00:46Z,dobleprofe,I don’t know if @Travistritt is taking request...,Twitter for iPhone,NaN,NaN,NaN,True,...,0,US,"Heber, UT",city,157,686,NaN,2015-08-10T14:32:42Z,False,en
2,1244776813366312960,492496769,2020-03-31T00:01:33Z,BrienKConvery,Tiny minuscule droplets of #Covid19 🦠 can last...,Twitter for iPhone,NaN,NaN,NaN,False,...,0,CA,"Toronto, Ontario",city,1697,4957,NaN,2012-02-14T19:55:35Z,False,en
3,1244776829925425153,861416960164286466,2020-03-31T00:01:37Z,NatKusendova,Do you have an old computer 💻 sitting around a...,Twitter for iPhone,NaN,NaN,NaN,False,...,5,CA,"Toronto, Ontario",city,4623,965,NaN,2017-05-08T03:06:22Z,True,en
4,1244776892395409408,1020933734642708485,2020-03-31T00:01:52Z,bennieagro,we pray God keeps us safe.. #BENNIE needs you ...,Twitter for iPhone,NaN,NaN,NaN,False,...,0,NG,"Rayfield, Nigeria",city,2079,110,NaN,2018-07-22T07:29:05Z,False,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1247041684950659072,30839743,2020-04-06T06:01:21Z,ecapobianco,@JeremyKonyndyk “The biggest variable in this ...,Twitter for iPhone,NaN,3.981727e+08,JeremyKonyndyk,False,...,1,CH,"Versoix, Suisse",city,2402,645,NaN,2009-04-13T09:56:49Z,True,en
9995,1247041883706200068,2241141325,2020-04-06T06:02:08Z,MukundShevare,Let's fight against Corona Virus. SAHAJAYOGA M...,Twitter for Android,1.247042e+18,2.241141e+09,MukundShevare,False,...,1,IN,"Nandgaon, India",city,40,37,NaN,2013-12-11T17:49:08Z,False,en
9996,1247041909102645248,3189460032,2020-04-06T06:02:14Z,narisashakt,#IndiaFightsCorona Know about the answers to ...,Twitter for Android,NaN,NaN,NaN,False,...,0,IN,"Meerut, India",city,59,153,NaN,2015-05-09T06:41:59Z,False,en
9997,1247041924940390401,80953698,2020-04-06T06:02:18Z,Naplestan,#prepper skills #CoronavirusPandemic https://...,Twitter for iPad,NaN,NaN,NaN,True,...,0,US,"Florida, USA",admin,20255,20620,NaN,2009-10-08T21:54:41Z,False,en


In [3]:
count_vect = CountVectorizer(stop_words="english")
X_train_counts = count_vect.fit_transform(df.text)
X_train_counts.shape

(9999, 38452)

In [4]:
count_vect.vocabulary_.get(u'algorithm')

3113

In [5]:
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(9999, 38452)

In [6]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(9999, 38452)

In [7]:
# target = []
df['from_US'] = 0
df.loc[df.country_code == 'US', 'from_US'] = 1
# for index, row in df.iterrows():
#     row['from_US'] = 1 if row['country_code'] == 'US' else 0
#     if row['country_code'] == 'US':
#         target.append(1)
#     else:
#         target.append(0)
        
df

,status_id,user_id,created_at,screen_name,text,source,reply_to_status_id,reply_to_user_id,reply_to_screen_name,is_quote,...,country_code,place_full_name,place_type,followers_count,friends_count,account_lang,account_created_at,verified,lang,from_US
0,1244776527499149315,80795043,2020-03-31T00:00:25Z,SimonWeaving,What colour is that virus? #coronavirus https:...,Twitter for iPhone,NaN,NaN,NaN,False,...,AU,"Newcastle, New South Wales",city,542,447,NaN,2009-10-08T07:50:31Z,False,en,0
1,1244776614900056064,3311560502,2020-03-31T00:00:46Z,dobleprofe,I don’t know if @Travistritt is taking request...,Twitter for iPhone,NaN,NaN,NaN,True,...,US,"Heber, UT",city,157,686,NaN,2015-08-10T14:32:42Z,False,en,1
2,1244776813366312960,492496769,2020-03-31T00:01:33Z,BrienKConvery,Tiny minuscule droplets of #Covid19 🦠 can last...,Twitter for iPhone,NaN,NaN,NaN,False,...,CA,"Toronto, Ontario",city,1697,4957,NaN,2012-02-14T19:55:35Z,False,en,0
3,1244776829925425153,861416960164286466,2020-03-31T00:01:37Z,NatKusendova,Do you have an old computer 💻 sitting around a...,Twitter for iPhone,NaN,NaN,NaN,False,...,CA,"Toronto, Ontario",city,4623,965,NaN,2017-05-08T03:06:22Z,True,en,0
4,1244776892395409408,1020933734642708485,2020-03-31T00:01:52Z,bennieagro,we pray God keeps us safe.. #BENNIE needs you ...,Twitter for iPhone,NaN,NaN,NaN,False,...,NG,"Rayfield, Nigeria",city,2079,110,NaN,2018-07-22T07:29:05Z,False,en,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1247041684950659072,30839743,2020-04-06T06:01:21Z,ecapobianco,@JeremyKonyndyk “The biggest variable in this ...,Twitter for iPhone,NaN,3.981727e+08,JeremyKonyndyk,False,...,CH,"Versoix, Suisse",city,2402,645,NaN,2009-04-13T09:56:49Z,True,en,0
9995,1247041883706200068,2241141325,2020-04-06T06:02:08Z,MukundShevare,Let's fight against Corona Virus. SAHAJAYOGA M...,Twitter for Android,1.247042e+18,2.241141e+09,MukundShevare,False,...,IN,"Nandgaon, India",city,40,37,NaN,2013-12-11T17:49:08Z,False,en,0
9996,1247041909102645248,3189460032,2020-04-06T06:02:14Z,narisashakt,#IndiaFightsCorona Know about the answers to ...,Twitter for Android,NaN,NaN,NaN,False,...,IN,"Meerut, India",city,59,153,NaN,2015-05-09T06:41:59Z,False,en,0
9997,1247041924940390401,80953698,2020-04-06T06:02:18Z,Naplestan,#prepper skills #CoronavirusPandemic https://...,Twitter for iPad,NaN,NaN,NaN,True,...,US,"Florida, USA",admin,20255,20620,NaN,2009-10-08T21:54:41Z,False,en,1


In [8]:
# training

In [9]:
#studing:)

clf = MultinomialNB().fit(X_train_tfidf, df.from_US)

docs_new = ["Great news for parents: The COVID-19 vaccine is approved for kids under 5. As appointments ramp up, you’ll be able to find a vaccine at pediatricians’ offices, pharmacies, and more. It's safe and effective—this is another big step to protect our loved ones and communities.", "Ney sou muito seu fa"]

X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s' % (doc, df.from_US[category]))

"Great news for parents: The COVID-19 vaccine is approved for kids under 5. As appointments ramp up, you’ll be able to find a vaccine at pediatricians’ offices, pharmacies, and more. It's safe and effective—this is another big step to protect our loved ones and communities." => 1
'Ney sou muito seu fa' => 0


In [10]:
text_clf = Pipeline([
    ('vect', CountVectorizer(stop_words="english")),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

text_clf.fit(df.text, df.from_US)

Pipeline(steps=[('vect', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])

In [11]:
df_test = pd.read_csv(opath+"test.csv", sep=';', on_bad_lines='skip')

df_test['from_US'] = 0
df_test.loc[df_test.country_code == 'US', 'from_US'] = 1

df_test
docs_test = df_test.text
predicted = text_clf.predict(docs_test)
print((predicted.shape))
print((df_test.from_US.shape))

np.mean(predicted == df_test.from_US)

(29999,)
(29999,)


0.7731591053035101

In [12]:
dump(text_clf, 'text_clf.joblib')

['text_clf.joblib']

In [13]:
clf = load('text_clf.joblib') 

In [15]:
clf.predict([ "Ney sou muito seu fa","Great news for parents: The COVID-19 vaccine is approved for kids under 5. As appointments ramp up, you’ll be able to find a vaccine at pediatricians’ offices, pharmacies, and more. It's safe and effective—this is another big step to protect our loved ones and communities."])

array([0, 1])